In [1]:
from dotenv import load_dotenv
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.core.tools import QueryEngineTool
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
load_dotenv()


# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

# ollama model
Settings.llm = Ollama(model="qwen2.5:7b", request_timeout=360.0)

# function tools
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b

multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

# rag pipeline
documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

# response = query_engine.query("What was the total amount of the 2023 Canadian federal budget?")
# print(response)

# rag pipeline as a tool
budget_tool =  QueryEngineTool.from_defaults(
    query_engine, 
    name="canadian_budget_2023",
    description="A RAG engine with some basic facts about the 2023 Canadian federal budget."
)

agent = ReActAgent.from_tools([multiply_tool, add_tool, budget_tool], verbose=True)

response = agent.chat("What is the total amount of the 2023 Canadian federal budget multiplied by 3? Go step by step, using a tool to do any math.")

print(response)

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

> Running step c7cd003d-dbd8-47e4-bd03-7f67e4957552. Step input: What is the total amount of the 2023 Canadian federal budget multiplied by 3? Go step by step, using a tool to do any math.
Thought: The user wants me to find out what's the total amount of the 2023 Canadian federal budget and then multiply it by 3. To get this information I need to use the `canadian_budget_2023` tool.
Action: canadian_budget_2023
Action Input: {'input': 'What is the total amount of the 2023 Canadian federal budget?'}
Observation: The total projected revenue for the 2023 Canadian federal budget is $456.8 billion, while the total projected expenditures are $496.9 billion. This results in a projected deficit of $40.1 billion.
> Running step 7286c477-01ad-4a85-b224-5a858aa4d38e. Step input: None
Thought: Now that I know the total projected expenditures for the 2023 Canadian federal budget, which is $496.9 billion, I can multiply this number by 3 to get the final answer.
Action: multiply
Action Input: {'a': 4